In [1]:
import time
import numpy as np

In [2]:
s = time.time()

In [3]:
def load_dataset(data_dir=''):
    """Load the train and test examples 
    """
    x_train = np.load("data/x_train.npy")
    y_train = np.load("data/y_train.npy")
    x_test = np.load("data/x_test.npy")
    y_test = np.load("data/y_test.npy")

    return x_train, y_train, x_test, y_test

In [4]:
x_train, y_train, x_test, y_test = load_dataset()

In [5]:
#x_train, y_train = x_train[:100], y_train[:100]

In [6]:
classes, counts = np.unique(y_train, return_counts=True)

In [7]:
def one_hotter(a, colors=256):
    #print(colors)
    b = np.zeros((a.size, colors))
    b[np.arange(a.size),a] = 1
    return b.T

In [9]:
def train(x_train, y_train):
    start = time.time()
    ans = np.zeros((10,256,784))
    for i, x in enumerate(x_train):
        y = y_train[i]
        hot = one_hotter(x)
        ans[y]+=hot
    print(time.time()-start)
    return ans

In [10]:
trained = train(x_train, y_train)

61.63002562522888


In [11]:
p_class = np.log(counts/sum(counts))
print(p_class)

[-2.30258509 -2.30258509 -2.30258509 -2.30258509 -2.30258509 -2.30258509
 -2.30258509 -2.30258509 -2.30258509 -2.30258509]


In [12]:
def p(pixel, val):
    count = trained[:, val, pixel]
    #total = counts[c]
    return np.log((count + 1)/(counts+1))

In [13]:
def p_total(x):
    totals = sum([p(pixel, col) for pixel, col in enumerate(x)])
    return totals + p_class

In [22]:
total_correct = 0
num_samples, num_classified_correctly = np.zeros((len(classes), 1)), np.zeros((len(classes), 1))
avg_accuracy = 0
conf_matrix = np.zeros((len(classes), len(classes)))
most_prototypical_ind, least_prototypical_ind = np.zeros((len(classes))), np.zeros((len(classes)))
most_prototypical_val, least_prototypical_val = np.ones((len(classes)))*-np.inf, np.ones((len(classes)))* np.inf
start = time.time()
correct, all_samples = 0,0
for i, x in enumerate(x_test):
    y = y_test[i]    
    p_tot = p_total(x)
    y_pred = np.argmax(p_tot)
    num_samples[y]+=1
    conf_matrix[y][y_pred]+=1
    
    
    if most_prototypical_val[y_pred] < p_tot[y_pred]:
        most_prototypical_ind[y_pred] = i
        most_prototypical_val[y_pred] = p_tot[y_pred]
    if least_prototypical_val[y_pred] > p_tot[y_pred]:
        least_prototypical_ind[y_pred] = i
        least_prototypical_val[y_pred] = p_tot[y_pred]
    
    if (y_pred == y):
        total_correct += 1
        num_classified_correctly[y] += 1
avg_classification_rate = total_correct/np.float(len(x_test))
avg_class_classification_rate = num_classified_correctly/num_samples
print("Avg Classification Rate", total_correct/np.float(len(x_test) ) )
print("Avg Class Classification Rate", avg_class_classification_rate )
print(conf_matrix)
    
    
print(correct, all_samples)
print(time.time()-start)

Avg Classification Rate 0.7421
Avg Class Classification Rate [[0.757]
 [0.907]
 [0.418]
 [0.852]
 [0.645]
 [0.772]
 [0.437]
 [0.918]
 [0.838]
 [0.877]]
[[757.   1.   9. 122.  23.   2.  73.   0.  13.   0.]
 [  3. 907.  14.  38.  21.   0.  17.   0.   0.   0.]
 [  7.   0. 418.  14. 329.   1. 219.   0.  12.   0.]
 [ 43.  12.   0. 852.  35.   1.  57.   0.   0.   0.]
 [  2.   2.  91. 105. 645.   2. 145.   1.   7.   0.]
 [  0.   0.   1.   0.   0. 772.   3. 181.   2.  41.]
 [153.   1.  87.  71. 230.   1. 437.   1.  19.   0.]
 [  0.   0.   0.   0.   0.  23.   0. 918.   0.  59.]
 [  0.   1.   4.  33.  13.  47.  26.  37. 838.   1.]
 [  0.   0.   0.   7.   0.  31.   0.  84.   1. 877.]]
0 0
37.43400979042053


In [23]:
print(most_prototypical_val, most_prototypical_ind)
least_prototypical_val, least_prototypical_ind

[-2113.76106845 -1126.6993192  -2246.70901891 -1240.35084962
 -2001.65871148  -804.80056521 -2220.20439304 -1130.4468316
 -1981.4900939  -1596.2356531 ] [2110.   10. 8591. 8510.   94.  839. 5172. 4945. 2351. 8139.]


(array([-3900.79126482, -2384.01281709, -4191.38949576, -2868.58657393,
        -3689.61302891, -3524.56169368, -4755.35691659, -2647.09351678,
        -4125.36914692, -3833.22856042]),
 array([2389., 7332.,  764., 2263., 6933., 2825., 3407., 5801., 8247.,
          50.]))

In [14]:
e = time.time()
print(e-s)

233.1182098388672
